In [3]:
#imports
import yaml
import cv2
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt

In [4]:
yaml_file_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/dataset.yaml'

with open(yaml_file_path, 'r') as file:
    dataset_info = yaml.safe_load(file)

image_width, image_height = 854, 480

def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_width, image_height))
    image = image / 255.0  #norm
    return image

train_images = []
train_labels = []

val_images = []
val_labels = []

train_img_path = os.path.join(dataset_info['path'], dataset_info['train'])
val_img_path = os.path.join(dataset_info['path'], dataset_info['val'])
train_lbl_path = os.path.join(dataset_info['path'], 'labels/train')
val_lbl_path = os.path.join(dataset_info['path'], 'labels/val')

label = 0
for filename in os.listdir(train_img_path):
    image_path = os.path.join(train_img_path, filename)
    image = read_and_preprocess_image(image_path)
    train_images.append(image)
for filename in os.listdir(val_img_path):
    image_path = os.path.join(val_img_path, filename)
    image = read_and_preprocess_image(image_path)
    val_images.append(image)
for filename in os.listdir(train_lbl_path):
    lbl_path = os.path.join(train_lbl_path, filename)
    with open(lbl_path, 'r') as file:
        line = file.readlines()[0]
        #print(line)
        splitted = line.split(' ')
        train_labels.append([float(splitted[1]), float(splitted[2]), float(splitted[3]), float(splitted[4][0:-2])])
for filename in os.listdir(val_lbl_path):
    lbl_path = os.path.join(val_lbl_path, filename)
    with open(lbl_path, 'r') as file:
        line = file.readlines()[0]
        #print(line)
        splitted = line.split(' ')
        val_labels.append([float(splitted[1]), float(splitted[2]), float(splitted[3]), float(splitted[4][0:-2])])



X_train = np.array(train_images)
y_train = np.array(train_labels)

X_val = np.array(val_images)
y_val = np.array(val_labels)


error: OpenCV(4.6.0) /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [18]:
len(X_train), len(y_train)
len(X_val), len(y_val)

(389, 389)

In [2]:
y_train

NameError: name 'y_train' is not defined

In [21]:
model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='elu', input_shape=(image_height, image_width, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='elu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='elu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='elu'),
        layers.MaxPooling2D((2, 2)),
        
        # Transpose convolution layers for upsampling
        layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='elu'),
        layers.Dropout(0.5),
        layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='elu'),
        layers.Dropout(0.5),
        layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='elu'),
        layers.Dropout(0.5),

        layers.Flatten(),
        layers.Dense(128, activation='elu'),
        layers.Dropout(0.5),
        layers.Dense(max_plates * 4)  # Output layer with (max_plates * 4) neurons for bounding box coordinates
    ])

model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [22]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/30
13/13 [==============================] - 226s 17s/step - loss: 0.1131 - accuracy: 0.9332 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
13/13 [==============================] - 198s 15s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
 1/13 [=>............................] - ETA: 3:07 - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(val_images,  val_labels, verbose=2)

In [11]:
model.save('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/Models/license_plate_detector.h5')

In [4]:
model = tf.keras.models.load_model('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/Models/license_plate_detector.h5')

In [14]:
image_path='/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/images/val/Cars406.png'
image = cv2.imread(image_path)
image = cv2.resize(image, (854, 480))
image = np.expand_dims(image, axis=0)

In [15]:
predictions = model.predict(image)
x, y, width, height = predictions[0]
x, y, width, height = int(x), int(y), int(width), int(height)
x, y, width, height

1/1 [==============================] - 0s 29ms/step


(0, 0, 0, 0)

In [ ]:
cv2.rectangle(image, (x, y), (x+width, y+height), (0, 255, 0), 2)
cv2.imshow('License Plate Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()